<a href="https://colab.research.google.com/github/pritam123-jain/Final-year/blob/master/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving movie_data.csv to movie_data.csv


In [ ]:
#url = 'https://raw.githubusercontent.com/pritam123-jain/Final-year/master/movie_dataset.csv'
df = pd.read_csv('movie_data.csv')

In [ ]:
df.head(2)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski


In [ ]:
df.shape # listing no of row and coloumn

(4803, 24)

In [ ]:
lst = [x.upper() for x in df.original_title] #listing all the title in upper case
print(lst)

['AVATAR', "PIRATES OF THE CARIBBEAN: AT WORLD'S END", 'SPECTRE', 'THE DARK KNIGHT RISES', 'JOHN CARTER', 'SPIDER-MAN 3', 'TANGLED', 'AVENGERS: AGE OF ULTRON', 'HARRY POTTER AND THE HALF-BLOOD PRINCE', 'BATMAN V SUPERMAN: DAWN OF JUSTICE', 'SUPERMAN RETURNS', 'QUANTUM OF SOLACE', "PIRATES OF THE CARIBBEAN: DEAD MAN'S CHEST", 'THE LONE RANGER', 'MAN OF STEEL', 'THE CHRONICLES OF NARNIA: PRINCE CASPIAN', 'THE AVENGERS', 'PIRATES OF THE CARIBBEAN: ON STRANGER TIDES', 'MEN IN BLACK 3', 'THE HOBBIT: THE BATTLE OF THE FIVE ARMIES', 'THE AMAZING SPIDER-MAN', 'ROBIN HOOD', 'THE HOBBIT: THE DESOLATION OF SMAUG', 'THE GOLDEN COMPASS', 'KING KONG', 'TITANIC', 'CAPTAIN AMERICA: CIVIL WAR', 'BATTLESHIP', 'JURASSIC WORLD', 'SKYFALL', 'SPIDER-MAN 2', 'IRON MAN 3', 'ALICE IN WONDERLAND', 'X-MEN: THE LAST STAND', 'MONSTERS UNIVERSITY', 'TRANSFORMERS: REVENGE OF THE FALLEN', 'TRANSFORMERS: AGE OF EXTINCTION', 'OZ: THE GREAT AND POWERFUL', 'THE AMAZING SPIDER-MAN 2', 'TRON: LEGACY', 'CARS 2', 'GREEN LANT

In [ ]:
df.insert(2,"Title",lst) #inserting new coloum with names in upper case from the list
#df.drop(columns="Title")

In [ ]:
df.head(2) # displaying top 2 dataset values

,index,budget,Title,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,AVATAR,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,PIRATES OF THE CARIBBEAN: AT WORLD'S END,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski


In [ ]:
df.columns # show coloum

Index(['index', 'budget', 'Title', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [ ]:
coloums = ['Title', 'cast', 'director', 'genres','vote_average','vote_count'] # filtering by requirement
df[coloums].head(2)

,Title,cast,director,genres,vote_average,vote_count
0,AVATAR,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,Action Adventure Fantasy Science Fiction,7.2,11800
1,PIRATES OF THE CARIBBEAN: AT WORLD'S END,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski,Adventure Fantasy Action,6.9,4500


In [ ]:
df[coloums].isnull().values.any() # checking null value if any = true else false 
#df1 = df.dropna()
#df1.head(1)

True

In [ ]:
C = df1['vote_average'].mean() # total votes average 
print(C)

6.318156424581003


In [ ]:
m = df1['vote_count'].quantile(0.8)
print(m)

2231.3999999999996


In [ ]:
filter = df1.copy().loc[df1['vote_count'] >= m]
filter.shape

(287, 25)

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)
#true Bayesian estimate = weighted rating
#R = average for the movie (mean) = (Rating)
#v = number of votes for the movie = (votes)
#m = minimum votes required to be listed in the Top 
#C = the mean vote across the whole report 

In [ ]:
filter['scores'] = filter.apply(weighted_rating, axis="columns")
filter['scores'].head()

0    7.059761
1    6.707124
2    6.306049
3    7.347711
5    6.060670
Name: scores, dtype: float64

In [ ]:
filter = filter.sort_values('scores', ascending=False)

#Print the movies
filter[['original_title', 'vote_count', 'vote_average', 'scores']].head(10)

,original_title,vote_count,vote_average,scores
662,Fight Club,9413,8.3,7.920222
65,The Dark Knight,12002,8.2,7.904979
96,Inception,13752,8.1,7.851242
3337,The Godfather,5893,8.4,7.828213
95,Interstellar,10867,8.1,7.796451
329,The Lord of the Rings: The Return of the King,8064,8.1,7.713808
1990,The Empire Strikes Back,5879,8.2,7.682252
262,The Lord of the Rings: The Fellowship of the Ring,8705,8.0,7.656846
2912,Star Wars,6624,8.1,7.651008
1818,Schindler's List,4329,8.3,7.625912
